<a href="https://colab.research.google.com/github/EricRoh-kr/st9_youtubevid/blob/main/st9_youtubevid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
# 필요한 라이브러리 설치
import subprocess
import sys
import os
import zipfile
import shutil
from datetime import datetime

# Google Colab 환경 확인 및 위젯 라이브러리 로드
try:
    from google.colab import files
    import ipywidgets as widgets
    from IPython.display import display, clear_output
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# --- 기존 핵심 기능 함수 (수정 거의 없음) ---

def install_ytdlp():
    """yt-dlp 라이브러리를 설치하거나 업그레이드합니다."""
    try:
        print("yt-dlp 설치/업그레이드 중...")
        # capture_output 대신 stdout/stderr 파이프를 사용하여 호환성 확보
        subprocess.check_call(
            [sys.executable, "-m", "pip", "install", "-U", "yt-dlp"],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )
        print("✅ yt-dlp가 준비되었습니다.")
        return True
    except subprocess.CalledProcessError as e:
        print(f"❌ yt-dlp 설치에 실패했습니다: {e.stderr}")
        return False

def create_download_folder():
    """다운로드 폴더를 생성하고, 이미 있다면 정리합니다."""
    folder_name = "영상저장함"
    if os.path.exists(folder_name):
        shutil.rmtree(folder_name)
    os.makedirs(folder_name)
    return folder_name

def download_videos(urls, quality, folder_name, output_widget):
    """지정된 비디오들을 다운로드하고 간소화된 진행 상황을 출력합니다."""
    with output_widget:
        print(f"⬇️ 다운로드를 시작합니다! (총 {len(urls)}개)")
        print("="*50)

        if quality == 'high':
            format_option = "bestvideo[ext=mp4]+bestaudio[ext=m4a]/best"
            print("✨ 고화질 모드로 다운로드합니다.")
        else:
            format_option = "best[height<=720]/best"
            print("👍 일반화질 모드로 다운로드합니다.")

        success_count = 0
        failed_urls = []

        for i, url in enumerate(urls, 1):
            if not url.strip():
                continue

            print(f"\n[{i}/{len(urls)}] 다운로드 중...", end="")

            try:
                cmd = [
                    "yt-dlp",
                    "-f", format_option,
                    "-o", f"{folder_name}/%(title)s.%(ext)s",
                    "--no-warnings",
                    "# '--progress' 옵션 제거로 로그 간소화",
                    url
                ]
                result = subprocess.run(cmd, capture_output=True, text=True, encoding="utf-8")

                if result.returncode == 0:
                    print(" ✅ 완료!")
                    success_count += 1
                else:
                    print(" ❌ 실패!")
                    failed_urls.append(url)

            except Exception as e:
                print(f" ❌ 처리 중 예외 발생: {e}")
                failed_urls.append(url)

        print("\n" + "="*50)
        print("🎉 다운로드 결과 🎉")
        print(f"✅ 성공: {success_count}개")
        print(f"❌ 실패: {len(failed_urls)}개")

        if failed_urls:
            print("\n❌ 실패한 링크:")
            for url in failed_urls:
                print(f"   - {url}")

        return success_count > 0

def create_and_download_zip(folder_name, output_widget):
    """폴더를 ZIP으로 압축하고 Colab에서 다운로드합니다."""
    with output_widget:
        today = datetime.now().strftime("%m%d")
        zip_filename = f"{today}_유튜브다운로드_{datetime.now().strftime('%H%M%S')}.zip"

        print("\n" + "="*50)
        print(f"🗜️ '{zip_filename}' 파일로 압축 중...")

        try:
            with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED, compresslevel=6) as zipf:
                for root, _, files_in_dir in os.walk(folder_name):
                    for file in files_in_dir:
                        file_path = os.path.join(root, file)
                        zipf.write(file_path, os.path.relpath(file_path, folder_name))

            print(f"✅ 압축 완료: {zip_filename}")
            print("⬇️ 브라우저에서 파일 다운로드를 시작합니다...")
            files.download(zip_filename)
            print("👍 다운로드 창이 나타나지 않으면 팝업 차단을 확인해주세요.")

        except Exception as e:
            print(f"❌ ZIP 파일 생성 또는 다운로드 중 오류 발생: {e}")


# --- ipywidgets UI 설정 및 이벤트 핸들러 ---

def on_download_button_clicked(b):
    """'다운로드 시작' 버튼 클릭 시 실행될 함수"""
    output_area.clear_output()

    urls = urls_input.value.strip().split('\n')
    urls = [url for url in urls if url.strip()] # 비어있는 라인 제거

    if not urls:
        with output_area:
            print("❌ 다운로드할 링크를 입력해주세요!")
        return

    quality = 'high' if quality_choice.value == '고화질 (최고 품질)' else 'normal'

    # yt-dlp 설치 확인
    with output_area:
        if not install_ytdlp():
            return

    folder_name = create_download_folder()

    has_downloads = download_videos(urls, quality, folder_name, output_area)

    if has_downloads:
        create_and_download_zip(folder_name, output_area)

# --- CLI 버전 (Colab이 아닐 때 사용) ---

def main_cli():
    """기존의 CLI 기반 메인 함수"""
    print("로컬 환경에서 CLI 모드로 실행합니다.")

    if not install_ytdlp():
        return

    # (기존 CLI 코드)
    # 이 부분은 ipywidgets를 사용하지 않는 환경을 위해 유지할 수 있습니다.
    # 현재는 Colab 전용으로 만들었으므로 간단한 메시지만 남깁니다.
    print("이 스크립트는 현재 Google Colab 환경에 최적화되어 있습니다.")
    print("로컬에서 사용하려면 코드 수정이 필요합니다.")


# --- 메인 실행 로직 ---

if __name__ == "__main__":
    if IN_COLAB:
        # 1. 위젯 생성
        style = {'description_width': 'initial'}
        urls_input = widgets.Textarea(
            placeholder='다운로드할 유튜브 링크를 한 줄에 하나씩 입력하세요.\n예시):\nhttps://www.youtube.com/watch?v=dQw4w9WgXcQ\nhttps://youtu.be/dQw4w9WgXcQ',
            layout=widgets.Layout(width='95%', height='150px')
        )
        quality_choice = widgets.RadioButtons(
            options=['일반화질 (720p 이하, 빠름)', '고화질 (최고 품질, 용량 큼)'],
            description='화질 선택:',
            style=style
        )
        download_button = widgets.Button(
            description='다운로드 시작',
            button_style='success',
            icon='download',
            layout=widgets.Layout(width='200px', height='40px')
        )
        output_area = widgets.Output(layout=widgets.Layout(width='95%', border='1px solid grey', padding='10px'))

        # 2. 버튼에 이벤트 핸들러 연결
        download_button.on_click(on_download_button_clicked)

        # 3. UI 레이아웃 구성 및 표시
        ui_title = widgets.HTML("<h1>🎬 YouTube 영상 다운로더 for Colab</h1>")
        ui_description = widgets.HTML("<p>여러 개의 링크를 붙여넣고 화질을 선택한 뒤, '다운로드 시작' 버튼을 누르세요. 완료되면 자동으로 압축 파일 다운로드가 시작됩니다.</p>")

        ui_layout = widgets.VBox([
            ui_title,
            ui_description,
            urls_input,
            quality_choice,
            download_button,
            widgets.HTML("<hr><h3>📜 진행 상황</h3>"),
            output_area
        ])

        display(ui_layout)
    else:
        main_cli()

In [31]:
# 필요한 라이브러리 설치
import subprocess
import sys
import os
import zipfile
import shutil
from datetime import datetime

# Google Colab 환경 확인 및 위젯/HTML 표시 라이브러리 로드
try:
    from google.colab import files
    import ipywidgets as widgets
    from IPython.display import display, clear_output
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# --- 실행 잠금 장치 (전역 변수) ---
# 클릭 이벤트가 중복 실행되는 것을 방지하기 위한 플래그
IS_RUNNING = False

# --- 핵심 기능 함수 ---

def install_ytdlp(output_widget):
    """yt-dlp 라이브러리를 설치하거나 업그레이드합니다."""
    with output_widget:
        try:
            print("yt-dlp 설치/업그레이드 중...")
            subprocess.check_call(
                [sys.executable, "-m", "pip", "install", "-U", "yt-dlp"],
                stdout=subprocess.PIPE, stderr=subprocess.PIPE
            )
            print("✅ yt-dlp가 준비되었습니다.")
            return True
        except subprocess.CalledProcessError:
            print("❌ yt-dlp 설치에 실패했습니다.")
            return False

def create_download_folder():
    """다운로드 폴더를 생성하고, 이미 있다면 정리합니다."""
    folder_name = "영상저장함"
    if os.path.exists(folder_name):
        shutil.rmtree(folder_name)
    os.makedirs(folder_name)
    return folder_name

def download_videos_to_folder(urls, quality, folder_name, output_widget):
    """영상들을 지정된 폴더에만 저장하고, 진행 상황을 출력합니다."""
    with output_widget:
        print(f"⬇️ 영상들을 폴더에 저장합니다... (총 {len(urls)}개)")
        print("="*50)

        if quality == '고화질 (최고 품질, 용량 큼)':
            format_option = "bestvideo[ext=mp4]+bestaudio[ext=m4a]/best"
            print("✨ 고화질 모드로 저장합니다.")
        else:
            format_option = "best[height<=720]/best"
            print("👍 일반화질 모드로 저장합니다.")

        success_count = 0
        failed_urls = []

        for i, url in enumerate(urls, 1):
            if not url.strip():
                continue

            print(f"\n[{i}/{len(urls)}] 처리 중: {url}", end="")
            try:
                output_template = os.path.join(folder_name, "%(title)s.%(ext)s")
                cmd = ["yt-dlp", "-f", format_option, "-o", output_template, "--no-warnings", url]
                result = subprocess.run(cmd, capture_output=True, text=True, encoding="utf-8")

                if result.returncode == 0:
                    print(" ✅ 저장 완료!")
                    success_count += 1
                else:
                    print(" ❌ 저장 실패!")
                    print(f"   [오류 원인]: {result.stderr.strip()}")
                    failed_urls.append(url)

            except Exception as e:
                print(f" ❌ 처리 중 예외 발생: {e}")
                failed_urls.append(url)

        print("\n" + "="*50)
        print("🎉 폴더 저장 결과 🎉")
        print(f"✅ 성공: {success_count}개")
        print(f"❌ 실패: {len(failed_urls)}개")

        if failed_urls:
            print("\n❌ 실패한 링크:")
            for url in failed_urls:
                print(f"   - {url}")

        return success_count > 0

def create_and_download_zip(folder_name, output_widget):
    """폴더를 ZIP으로 압축하고 Colab에서 자동 다운로드합니다."""
    with output_widget:
        today = datetime.now().strftime("%m%d")
        zip_filename = f"{today}_유튜브다운로드_{datetime.now().strftime('%H%M%S')}.zip"

        print("\n" + "="*50)
        print(f"🗜️ '{zip_filename}' 파일로 압축 중...")

        try:
            with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED, compresslevel=6) as zipf:
                for root, _, files_in_dir in os.walk(folder_name):
                    for file in files_in_dir:
                        file_path = os.path.join(root, file)
                        zipf.write(file_path, os.path.relpath(file_path, folder_name))

            print(f"✅ 압축 완료: {zip_filename}")
            print("⬇️ 브라우저에서 파일 다운로드를 시작합니다...")
            files.download(zip_filename)
            print("👍 다운로드 창이 나타나지 않으면 팝업 차단을 확인해주세요.")

        except Exception as e:
            print(f"❌ ZIP 파일 생성 또는 다운로드 중 오류 발생: {e}")

# --- ipywidgets UI 및 이벤트 핸들러 ---

def on_download_button_clicked(b):
    """'다운로드 시작' 버튼 클릭 시 실행될 완전 자동화 함수"""
    global IS_RUNNING
    if IS_RUNNING:
        with output_area:
            print("이미 작업이 진행 중입니다. 완료될 때까지 기다려주세요.")
        return

    IS_RUNNING = True
    download_button.disabled = True
    output_area.clear_output()

    try:
        with output_area:
            urls = urls_input.value.strip().split('\n')
            urls = [url for url in urls if url.strip()]

            if not urls:
                print("❌ 다운로드할 링크를 입력해주세요!")
                return

            if not install_ytdlp(output_area):
                return

            folder_name = create_download_folder()
            has_downloads = download_videos_to_folder(urls, quality_choice.value, folder_name, output_area)

            if has_downloads:
                create_and_download_zip(folder_name, output_area)
            else:
                print("\n❌ 저장된 영상이 없어 압축을 진행하지 않습니다.")
    finally:
        # 모든 작업이 끝나면 잠금 해제
        IS_RUNNING = False
        download_button.disabled = False

# --- 메인 실행 로직 ---

if __name__ == "__main__":
    if IN_COLAB:
        style = {'description_width': 'initial'}
        urls_input = widgets.Textarea(
            placeholder='다운로드할 유튜브 링크를 한 줄에 하나씩 입력하세요.',
            layout=widgets.Layout(width='95%', height='150px')
        )
        quality_choice = widgets.RadioButtons(
            options=['일반화질 (720p 이하, 빠름)', '고화질 (최고 품질, 용량 큼)'],
            description='화질 선택:', style=style
        )
        download_button = widgets.Button(
            description='다운로드 시작',
            button_style='success', icon='download',
            layout=widgets.Layout(width='200px', height='40px')
        )
        output_area = widgets.Output(layout=widgets.Layout(width='95%', border='1px solid grey', padding='10px'))

        download_button.on_click(on_download_button_clicked)

        ui_title = widgets.HTML("<h1>🎬 YouTube 영상 다운로더</h1>")
        ui_description = widgets.HTML("<p>링크를 붙여넣고 '다운로드 시작' 버튼을 누르세요. 완료되면 자동으로 압축 파일 다운로드가 시작됩니다.</p>")

        display(widgets.VBox([
            ui_title, ui_description, urls_input, quality_choice,
            download_button, widgets.HTML("<hr><h3>📜 진행 상황</h3>"), output_area
        ]))
    else:
        print("이 스크립트는 Google Colab 환경에서만 작동합니다.")